# DCGAN con CelebA (caras) — Ejercicio 2

- Generación de imágenes con **DCGAN** usando **CelebA**.
- Guarda muestras, curvas de pérdidas y actualiza `results/summary.csv`.
- (Opcional) Calcula **FID** si hay acceso a los pesos de InceptionV3.

In [ ]:

# === Imports y setup ===
import os, time, numpy as np, tensorflow as tf

from data import GANCelebAData
from models import build_dcgan_generator, build_dcgan_discriminator, build_dcgan_gan
from train import DCGANTrainer
from utils import (
    plot_gan_training_history, save_gan_results,
    generate_final_gan_samples, update_summary_csv, calculate_fid,
)

NOTEBOOK = "2_dcgan_celeba"
MODEL_NAME = "DCGAN(CelebA-64)"
LATENT_DIM = 100

# Cambiá este path a donde tengas las imágenes de CelebA (por ejemplo: "/data/celeba/img_align_celeba")
CELEBA_DIR = r"./assets/img_align_celeba"
assert os.path.exists(CELEBA_DIR), "Asegúrate de apuntar CELEBA_DIR a la carpeta con las imágenes de CelebA"

# Dataset real (CelebA) → 64x64, [-1,1]
data_loader = GANCelebAData(dataset_dir=CELEBA_DIR, image_size=64, batch_size=64, take=50000)
dataset = data_loader.create_dataset()
# (opcional) ver muestras reales:
# data_loader.show_sample()


ModuleNotFoundError: No module named 'data'

In [ ]:

# === Construcción de modelos DCGAN ===
generator = build_dcgan_generator(latent_dim=LATENT_DIM)
discriminator = build_dcgan_discriminator(image_size=64)
gan = build_dcgan_gan(generator, discriminator)

trainer = DCGANTrainer(generator, discriminator, gan, latent_dim=LATENT_DIM)
trainer.compile_models(learning_rate=0.0002)


In [ ]:

# === Entrenamiento ===
history, train_time = trainer.train(
    dataset,
    epochs=50,            # ajusta según tus recursos
    save_interval=10,
    notebook_name=NOTEBOOK
)

# Curvas de entrenamiento
plot_gan_training_history(history, notebook_name=NOTEBOOK, model_name=MODEL_NAME)


In [ ]:

# === Generación de muestras y guardado del generador ===
generate_final_gan_samples(generator, LATENT_DIM, notebook_name=NOTEBOOK, num_samples=25)


In [ ]:

# === Guardar resultados del entrenamiento de la GAN ===
model_params = generator.count_params() + discriminator.count_params()
results = save_gan_results(
    history=history,
    training_time=float(train_time),
    model_name=MODEL_NAME,
    notebook_name=NOTEBOOK,
    epochs=len(history["d_loss"]),
    model_params=int(model_params),
)


In [ ]:

# === (Opcional) FID si hay pesos de InceptionV3 disponibles ===
def _gather_real_images(ds, n=500):
    xs = []
    for batch in ds:
        xs.append(batch.numpy())
        if sum(len(b) for b in xs) >= n:
            break
    X = np.concatenate(xs, axis=0)[:n]
    return X

def _sample_generated(G, zdim, n=500):
    z = tf.random.normal([n, zdim])
    fake = G(z, training=False).numpy()
    return fake

try:
    n_fid = 500
    real_imgs = _gather_real_images(dataset, n=n_fid)
    fake_imgs = _sample_generated(generator, LATENT_DIM, n=n_fid)

    fid_score = float(calculate_fid(real_imgs, fake_imgs, image_size=299))
    print(f"FID = {fid_score:.2f}")
    results["FID_score"] = fid_score
except Exception as e:
    print("No se pudo calcular FID (probablemente sin pesos de InceptionV3 o sin internet).")
    print("Detalle:", repr(e))
    results["FID_score"] = "N/A"

update_summary_csv(results, file_path="../results/summary.csv")
